In [3]:
import json
import os
# Path to COCO annotation file and output folder for YOLO labels
coco_ann_file = r"C:/Users/Admin/Desktop/infosys/coco2017_subset/annotation_subset/instances_val2017_subset.json"
output_dir = "C:/Users/Admin/Desktop/infosys/coco2017_subset/val2017"
os.makedirs(output_dir, exist_ok=True)
# Load COCO annotations
with open(coco_ann_file, 'r') as f:
    coco_data = json.load(f)
    # For example, if valid categories are 1, 2, 3, 5, 6, ... then you might have:
coco_categories = sorted([cat['id'] for cat in coco_data['categories']])
id_mapping = {orig_id: new_id for new_id, orig_id in enumerate(coco_categories)}
# Build a mapping from image_id to image info
img_info = {img['id']: img for img in coco_data['images']}
 #Process annotations per image
annotations_by_image = {}
for ann in coco_data['annotations']:
    img_id = ann['image_id']
    img = img_info[img_id]
    img_width, img_height = img['width'], img['height']

    x, y, w, h = ann['bbox']
    # Convert COCO bbox [x, y, w, h] to YOLO format [x_center, y_center, width, height] normalized
    x_center = (x + w / 2) / img_width
    y_center = (y + h / 2) / img_height
    norm_w = w / img_width
    norm_h = h / img_height

    # Remap the category id
    orig_cat = ann['category_id']
    if orig_cat not in id_mapping:
        continue  # skip if the category is not in the mapping
    new_cat = id_mapping[orig_cat]

    if img_id not in annotations_by_image:
        annotations_by_image[img_id] = []
    annotations_by_image[img_id].append(f"{new_cat} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}")
    # Write YOLO label files for each image
for img_id, ann_list in annotations_by_image.items():
    file_name = img_info[img_id]['file_name']
    base_name = os.path.splitext(file_name)[0]
    # Save the label file in the same directory as your images, or wherever you prefer
    out_file = os.path.join(output_dir, base_name + '.txt')
    with open(out_file, 'w') as f:
        for line in ann_list:
            f.write(line + '\n')